In [1]:
import pandas as pd
import numpy as np
import pymysql
import sqlalchemy as alch
import os
from dotenv import load_dotenv
from getpass import getpass

## I. DATA EXPLORATION AND CLEANING FOR STREAMING SERVICES:

In [2]:
file_path_netflix = '/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/netflix_titles.csv'
netflix = pd.read_csv(file_path_netflix)
netflix.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


1. Adding a column to specify the streaming service, as I will be using two more later.

In [3]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


#### * The columns that I care cleaning are the following: Country, date_added, rating, duration and listed in

In [4]:
# Add a new column to differenciate streaming_service as I will add a couple more later for some coparisson analysis
netflix['streaming_service'] = 'Netflix'
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,streaming_service
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Netflix
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Netflix
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Netflix
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Netflix
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Netflix


## 1. Handling Null Values and Cleaning "Country" column (doing this just for Netflix):

In [5]:
import requests
from bs4 import BeautifulSoup

def fetch_country_information(title):
    url = f'https://en.wikipedia.org/wiki/{title.replace(" ", "_")}'  # Adjust URL format for Wikipedia
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Find the section or tag containing the country of production
        country_info = soup.find('th', text='Country')
        if country_info:
            country = country_info.find_next('td').text.strip()
            return country
    return None  # Return None if country information isn't found or request fails

# Assuming 'titles_df' is your DataFrame with titles and a 'country' column that contains null values
for index, row in netflix.iterrows():
    if pd.isnull(row['country']):
        title = row['title']  # Assuming 'title' is the column name with movie/TV show titles
        country = fetch_country_information(title)
        if country:
            netflix.at[index, 'country'] = country


/var/folders/8n/dc8xfymj17s08w7814ggqp9w0000gn/T/ipykernel_75835/3692986262.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  country_info = soup.find('th', text='Country')


In [6]:
# Display the updated DataFrame
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   show_id            8807 non-null   object
 1   type               8807 non-null   object
 2   title              8807 non-null   object
 3   director           6173 non-null   object
 4   cast               7982 non-null   object
 5   country            8069 non-null   object
 6   date_added         8797 non-null   object
 7   release_year       8807 non-null   int64 
 8   rating             8803 non-null   object
 9   duration           8804 non-null   object
 10  listed_in          8807 non-null   object
 11  description        8807 non-null   object
 12  streaming_service  8807 non-null   object
dtypes: int64(1), object(12)
memory usage: 894.6+ KB


In [7]:
# Load an IMBD dataset for complementing country data
netflix_complement_data_imbd = pd.read_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/netflix_complement_data_imbd.csv')
netflix_complement_data_imbd.sample(2)

,Unnamed: 0,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,...,averageRating,numVotes,ordering,title,region,language,types,attributes,isOriginalTitle,Description
2502,2502,3519,tt13510660,movie,Dhamaka,Dhamaka,0,2021,\N,103,...,7.1,41916,22,Dhamaka,IN,en,imdbDisplay,\N,0,When a cynical ex-TV news anchor gets an alarm...
3201,3201,4385,tt1742336,movie,Your Sister's Sister,Your Sister's Sister,0,2011,\N,90,...,6.7,27971,21,Your Sister's Sister,IN,en,imdbDisplay,\N,0,A man (Mark Duplass) falls into bed with his b...


In [8]:
# Adding more countries from the IMBD dataset 
for index, row in netflix.iterrows():
    if pd.isna(row['country']):
        # Find a match in the complement dataset
        match = netflix_complement_data_imbd[netflix_complement_data_imbd['primaryTitle'] == row['title']]
        if not match.empty:
            # If a match is found and 'Country Availability' is not null, update 'country' in the main dataset
            if not pd.isna(match.iloc[0]['region']):
                netflix.at[index, 'country'] = match.iloc[0]['region']
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   show_id            8807 non-null   object
 1   type               8807 non-null   object
 2   title              8807 non-null   object
 3   director           6173 non-null   object
 4   cast               7982 non-null   object
 5   country            8121 non-null   object
 6   date_added         8797 non-null   object
 7   release_year       8807 non-null   int64 
 8   rating             8803 non-null   object
 9   duration           8804 non-null   object
 10  listed_in          8807 non-null   object
 11  description        8807 non-null   object
 12  streaming_service  8807 non-null   object
dtypes: int64(1), object(12)
memory usage: 894.6+ KB


In [9]:
# Looking the format of the countries from the IMBD dataset
two_char_countries = netflix[netflix['country'].str.len() == 2]

# Display the results
two_char_countries['country'].unique()

array(['IN', 'US', 'CA'], dtype=object)

In [35]:
# Replace specific values in the 'country' column
netflix['country'] = netflix['country'].replace({'IN': 'India', 'US': 'United States', 'U.S.': 'United States', 'CA': 'Canada'})

In [11]:
# Save the updated DataFrame back to a CSV file
netflix.to_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/netflix_titles_updated.csv', index=False)

## 2. Concat Dataframes: Netflix + Disney + Prime Video:

In [12]:
file_path_prime = '/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/amazon_prime_titles.csv'
prime_df = pd.read_csv(file_path_prime)
prime_df['streaming_service'] = 'Prime Video'
prime_df.sample(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,streaming_service
9293,s9294,Movie,The Breaking Point,James Hunter,"Diana Lovell, Erik Grey, Sean Nelson, Reggie P...",NaN,NaN,2021,16+,73 min,"Action, Drama, Suspense","Shawn Dickerson, a street guy is accused of ki...",Prime Video


In [32]:
prime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   show_id            9668 non-null   object
 1   type               9668 non-null   object
 2   title              9668 non-null   object
 3   director           7585 non-null   object
 4   cast               8435 non-null   object
 5   country            672 non-null    object
 6   date_added         155 non-null    object
 7   release_year       9668 non-null   int64 
 8   rating             9331 non-null   object
 9   duration           9668 non-null   object
 10  listed_in          9668 non-null   object
 11  description        9668 non-null   object
 12  streaming_service  9668 non-null   object
dtypes: int64(1), object(12)
memory usage: 982.0+ KB


In [13]:
file_path_disney = '/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/disney_plus_titles.csv'
disney_df = pd.read_csv(file_path_disney)
disney_df['streaming_service'] = 'Disney'
disney_df.sample(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,streaming_service
849,s850,TV Show,Disney's Recess,NaN,"Rickey Collins, Jason Davis, Ashley Johnson, R...",United States,"November 12, 2019",1997,TV-Y7,3 Seasons,"Animation, Comedy, Kids",A portrayal of life at Third Street Elementary...,Disney


In [33]:
disney_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   show_id            1450 non-null   object
 1   type               1450 non-null   object
 2   title              1450 non-null   object
 3   director           977 non-null    object
 4   cast               1260 non-null   object
 5   country            1231 non-null   object
 6   date_added         1447 non-null   object
 7   release_year       1450 non-null   int64 
 8   rating             1447 non-null   object
 9   duration           1450 non-null   object
 10  listed_in          1450 non-null   object
 11  description        1450 non-null   object
 12  streaming_service  1450 non-null   object
dtypes: int64(1), object(12)
memory usage: 147.4+ KB


In [14]:
# Concat the streaming services dataframes (they all have the same columns)
df = pd.concat([netflix, prime_df, disney_df], ignore_index=True)

In [15]:
# Remove non-numeric characters from 'show_id'as this are the same for the 3 datasets
df['show_id'] = df['show_id'].str.extract('(\d+)')

# Add prefix to 'show_id' based on 'streaming_service'
df['show_id'] = df.apply(
    lambda row: ('n' if row['streaming_service'] == 'Netflix' else 
                 'd' if row['streaming_service'] == 'Disney' else 
                 'p' if row['streaming_service'] == 'Prime Video' else '') + row['show_id'],
    axis=1
)

In [16]:
# Country cleaning for the concat dataframe: 
#Strip leading/trailing spaces
df['country'] = df['country'].str.strip()

# Replace commas at the beginning or end of the string
df['country'] = df['country'].str.replace(r'^,|,$', '', regex=True)

## 3. Handling Null Values and Cleaning "Date_added" column:

In [19]:
# Convert 'date_added' to datetime, coerce errors to NaT
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Fill NaT values with a default date or handle them as needed
# Example: Filling with a specific date (you can choose a suitable date)
df['date_added'].fillna(pd.Timestamp('2000-01-01'), inplace=True)

# Extract month and year
df['month'] = df['date_added'].dt.month
df['year'] = df['date_added'].dt.year


In [20]:
# Define a function to assign quarter
def assign_quarter(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'

In [21]:
# Apply the function to create the 'quarter' column
df['quarter'] = df['year'].astype(str) + df['month'].apply(assign_quarter)

## 4. Cleaning the Rating column:

In [22]:
# Move 'min' values to the duration column:
df.loc[df['rating'].str.contains('min', na=False), 'duration'] = df['rating']
df.loc[df['rating'].str.contains('min', na=False), 'rating'] = None
# Group the ratings in 4 categories 
replacements = {
    'TV-G': 'Kids (All)', 'TV-Y': 'Kids (All)', 'ALL': 'Kids (All)', 'ALL_AGES': 'Kids (All)', 'G': 'Kids (All)',
    '7+': 'Older Kids (7+)', 'TV-Y7': 'Older Kids (7+)', 'TV-Y7-FV': 'Older Kids (7+)', 'TV-PG': 'Older Kids (7+)', 'PG': 'Older Kids (7+)',
    '13+': 'Teens (13+)', 'PG-13': 'Teens (13+)',
    'TV-14': 'Young Adults (16+)', '16+': 'Young Adults (16+)', 'AGES_16_': 'Young Adults (16+)',
    'NC-17': 'Adults (18+)', 'NR': 'Adults (18+)', 'UNRATED': 'Adults (18+)', 'R': 'Adults (18+)', 'NOT_RATE': 'Adults (18+)', 'TV-NR': 'Adults (18+)','UR': 'Adults (18+)', '18+': 'Adults (18+)', 'AGES_18_': 'Adults (18+)', 'TV-MA': 'Adults (18+)'
}

df['rating'] = df['rating'].replace(replacements)
df.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,streaming_service,month,year,quarter
0,n1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,Teens (13+),90 min,Documentaries,"As her father nears the end of his life, filmm...",Netflix,9,2021,2021Q3
1,n2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,Adults (18+),2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Netflix,9,2021,2021Q3
2,n3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",India,2021-09-24,2021,Adults (18+),1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Netflix,9,2021,2021Q3
3,n4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,Adults (18+),1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Netflix,9,2021,2021Q3
4,n5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,Adults (18+),2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Netflix,9,2021,2021Q3


## 5. Cleaning the Duration column:

In [23]:
import numpy as np
import math

# Function to round up to nearest 10 so I can control better this data and create better analysis and visualization
def round_up_to_nearest_10(x):
    return int(math.ceil(x / 10.0)) * 10

In [24]:
# Apply function so that changes are only for movies (duratin in "min") and not for TV shows (seasons)
df.loc[df['duration'].str.contains('min'), 'duration'] = (
df[df['duration'].str.contains('min')]['duration']
.str.replace(' min', '')
.astype(int)
.apply(round_up_to_nearest_10)
    )

In [26]:
# Save the updated DataFrame back to a CSV file
df.to_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/netflix_titles_updated.csv', index=False)

## 6. Adding information about Netflix Original Productions

In [ ]:
# Web scrapping Wikipedia to obtain the list of Netflix Originals for Movies and Tv Shows

import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of Wikipedia pages to scrape
urls = [
        'https://en.wikipedia.org/wiki/List_of_Netflix_original_programming',
    'https://en.wikipedia.org/wiki/List_of_ended_Netflix_original_programming',
    'https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2015%E2%80%932017)',
    'https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2018)',
    'https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2019)',
    'https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2020)',
    'https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2021)',
    'https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(2022)',
    'https://en.wikipedia.org/wiki/List_of_Netflix_original_films_(since_2023)'
]

# Initialize an empty DataFrame to store the extracted data
netflix_data = pd.DataFrame(columns=['title', 'type'])

# Loop through each URL
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title_selector = 'table.wikitable td:first-child i'  # Assuming titles are always inside <i> tags
    titles = [element.get_text(strip=True) for element in soup.select(title_selector)]
    temp_data = pd.DataFrame(columns=['title', 'type'])

    # Determine the type based on the URL
    if url in ['https://en.wikipedia.org/wiki/List_of_Netflix_original_programming',
               'https://en.wikipedia.org/wiki/List_of_ended_Netflix_original_programming']:
        type_value = 'TV Show'
    else:
        type_value = 'Movie'

    # Add the extracted titles to the DataFrame
    for title in titles:
        temp_df = pd.DataFrame([{'title': title, 'type': type_value}])
        temp_data = pd.concat([temp_data, temp_df], ignore_index=True)
    
    # Concatenate temporary DataFrame to main DataFrame
    netflix_data = pd.concat([netflix_data, temp_data], ignore_index=True)

In [ ]:
# Save the updated DataFrame back to a CSV file
netflix_data.to_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/netflix_originals.csv', index=False)

In [27]:
# Load main dataset
df = pd.read_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/netflix_titles_updated.csv')

# Load the Netflix originals dataset
netflix_originals = pd.read_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/netflix_originals.csv')

# Create a set of Netflix original titles for faster lookup
original_titles = set(netflix_originals['title'])

# Add a new column to your main dataset to indicate Netflix originals
df['is_netflix_original'] = df['title'].isin(original_titles)

In [28]:
# Save the updated DataFrame to a CSV named Streaming Services 
df.to_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/streaming_services_df.csv', index=False)

## 7. Adding Revenue and Subscribers columns

In [29]:
# Creating a financial dataset according to one found in Kaggle
financial_data = {
    "year": [2011, 2011, 2011, 2011, 2012, 2012, 2012, 2012, 2013, 2013, 2013, 2013,
             2014, 2014, 2014, 2014, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016,
             2017, 2017, 2017, 2017, 2018, 2018, 2018, 2018, 2019, 2019, 2019, 2019,
             2020, 2020, 2020, 2020, 2021, 2021, 2021, 2021, 2022, 2022, 2022, 2022,
             2023, 2023, 2023],
    "quarter": ["2011Q1", "2011Q2", "2011Q3", "2011Q4", "2012Q1", "2012Q2", "2012Q3", "2012Q4",
                "2013Q1", "2013Q2", "2013Q3", "2013Q4", "2014Q1", "2014Q2", "2014Q3", "2014Q4",
                "2015Q1", "2015Q2", "2015Q3", "2015Q4", "2016Q1", "2016Q2", "2016Q3", "2016Q4",
                "2017Q1", "2017Q2", "2017Q3", "2017Q4", "2018Q1", "2018Q2", "2018Q3", "2018Q4",
                "2019Q1", "2019Q2", "2019Q3", "2019Q4", "2020Q1", "2020Q2", "2020Q3", "2020Q4",
                "2021Q1", "2021Q2", "2021Q3", "2021Q4", "2022Q1", "2022Q2", "2022Q3", "2022Q4",
                "2023Q1", "2023Q2", "2023Q3"],
    "revenue": [719, 789, 822, 876, 870, 889, 905, 945, 1024, 1069, 1106, 1175,
                1270, 1340, 1409, 1485, 1573, 1645, 1738, 1823, 1958, 2105, 2290, 2478,
                2637, 2785, 2985, 3286, 3701, 3907, 3999, 4187, 4521, 4923, 5245, 5467,
                5768, 6148, 6436, 6644, 7163, 7342, 7483, 7709, 7868, 7970, 7926, 7852,
                8162, 8187, 8542],
    "subscribers": [None, None, 21, 22, 24, 26, 27, 30, 34, 36, 38, 41, 46, 48, 51, 54, 60, 63, 66, 71,
                    78, 80, 83, 89, 94, 99, 104, 111, 119, 124, 130, 139, 149, 152, 158, 167, 183, 193,
                    195, 204, 208, 209, 214, 222, 222, 221, 223, 231, 232, 238, 247]
}

# Create the DataFrame
netflix_financial_df = pd.DataFrame(financial_data)

In [30]:
# Merge the main DataFrame with the financial DataFrame
df = pd.merge(df, netflix_financial_df[['quarter', 'revenue', 'subscribers']], 
              on='quarter', 
              how='left')

df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,streaming_service,month,year,quarter,is_netflix_original,revenue,subscribers
0,n1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,Teens (13+),90,Documentaries,"As her father nears the end of his life, filmm...",Netflix,9,2021,2021Q3,True,7483.0,214.0
1,n2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,Adults (18+),2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Netflix,9,2021,2021Q3,True,7483.0,214.0


In [31]:
# Save the updated DataFrame to a CSV named Streaming Services 
df.to_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/streaming_services_df.csv', index=False)

## 8. Creating a Dataset for Top 10 Movies and TV shows of all times in Netflix to add some interesting facts in the visualization

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.netflix.com/tudum/top10/most-popular'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table
table = soup.find('table', class_='w-full mx-auto text-lg md:w-11/12 sm:text-xl md:text-2xl')

# Extract table headers
headers = [header.text for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    rows.append([cell.text.strip() for cell in row.find_all('td')])

# Create DataFrame
df_top_english_films = pd.DataFrame(rows, columns=headers)


# Add the 'top_10' column with 'films_english' as its value for all rows
df_top_english_films['top_10'] = 'films in english'

df_top_english_films

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.netflix.com/tudum/top10/most-popular/films-non-english'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table
table = soup.find('table', class_='w-full mx-auto text-lg md:w-11/12 sm:text-xl md:text-2xl')

# Extract table headers
headers = [header.text for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    rows.append([cell.text.strip() for cell in row.find_all('td')])

# Create DataFrame
df_top_non_english_films = pd.DataFrame(rows, columns=headers)


# Add the 'top_10' column with 'films_english' as its value for all rows
df_top_non_english_films['top_10'] = 'films (non-english)'

df_top_non_english_films

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.netflix.com/tudum/top10/most-popular/tv'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table
table = soup.find('table', class_='w-full mx-auto text-lg md:w-11/12 sm:text-xl md:text-2xl')

# Extract table headers
headers = [header.text for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    rows.append([cell.text.strip() for cell in row.find_all('td')])

# Create DataFrame
df_top_english_tv = pd.DataFrame(rows, columns=headers)


# Add the 'top_10' column with 'films_english' as its value for all rows
df_top_english_tv['top_10'] = 'tv in english'

df_top_english_tv

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.netflix.com/tudum/top10/most-popular/tv-non-english'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table
table = soup.find('table', class_='w-full mx-auto text-lg md:w-11/12 sm:text-xl md:text-2xl')

# Extract table headers
headers = [header.text for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    rows.append([cell.text.strip() for cell in row.find_all('td')])

# Create DataFrame
df_top_non_english_tv = pd.DataFrame(rows, columns=headers)


# Add the 'top_10' column with 'films_english' as its value for all rows
df_top_non_english_tv['top_10'] = 'tv (non-english)'

df_top_non_english_tv

In [ ]:
df_top_ten = pd.concat([df_top_english_films, df_top_non_english_films, df_top_english_tv ,df_top_non_english_tv], ignore_index=True)
# Save the updated DataFrame back to a CSV file
df_top_ten.to_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/netflix_top_ten.csv', index=False)

In [ ]:
# Load netflix_top_10 dataset
df_top_ten = pd.read_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/netflix_top_ten.csv')

In [ ]:
# Add a 'year' column with NaN as default
df_top_ten['year'] = np.nan

# Update the 'year' based on the condition
for index, row in df_top_ten.iterrows():
    title = row['Title']
    if title in df['title'].values:
        year = df[df['title'] == title]['year'].values[0]
        df_top_ten.at[index, 'year'] = year
    else:
        df_top_ten.at[index, 'year'] = 'after 2021'

df_top_ten

In [ ]:
# Save the updated DataFrame if needed
df_top_ten.to_csv('/Users/mairagutierrez/Documents/Ironhack/PROJECTS/project-IV--STREAMING-SERVICES-ANALYSIS/data/updated_netflix_top_ten.csv', index=False)